In [34]:
import os
import glob

import numpy as np
import nibabel as nib
from tqdm import tqdm


In [ ]:
data_dir = os.path.join('..', '..', 'data', 'preprocessed', 'Mathijs', 'targets USZ')
labels_tr_dir = os.path.join(data_dir, 'labelsTr', '*.nii.gz')
labels_ts_dir = os.path.join(data_dir, 'labelsTs', '*.nii.gz')

In [40]:
file_paths_train = glob.glob(labels_tr_dir)
file_paths_test = glob.glob(labels_ts_dir)

In [41]:
def binarize_segmentation(img_np, only_specific_vessel_class: int = None):
    if not only_specific_vessel_class:
        img_np = np.where(img_np >= 1, 1, 0)
    else:
        img_np = np.where(img_np == only_specific_vessel_class, 1, 0)
    return img_np

In [44]:
def transform_label_vols_in_dir(filepaths: list, output_dir: str, transformation: callable):
    os.makedirs(output_dir, exist_ok=True)
    for file_path in tqdm(filepaths):
        img_name = os.path.basename(file_path)
        img = nib.load(file_path)
        img_np = img.get_fdata()
        img_np = transformation(img_np)
        img = nib.Nifti1Image(img_np, img.affine, img.header)
        nib.save(img, os.path.join(output_dir, img_name))

In [65]:
# collapse all classes except aneurysm into one class
def collapse_all_classes_except_one(img_np, class_not_to_collapse: int):
    # Collapse all clasess to the class 1 except the class_not_to_collapse
    img_np = np.where((img_np != class_not_to_collapse) & (img_np != 0), 1.0, img_np)
    
    # Set the class_not_to_collapse class to 2
    img_np = np.where(img_np == class_not_to_collapse, 2, img_np)
    
    return img_np.astype(np.int8)

In [63]:
test_img = nib.load(file_paths_train[0])
test_img_np = test_img.get_fdata()

np.unique(collapse_all_classes_except_aneurysm(test_img_np).astype(np.int8))

array([0, 1, 2], dtype=int8)

# Task 1: binary segmentation: all vessels types vs background

In [45]:
transform_label_vols_in_dir(
    file_paths_train, 
    os.path.join(data_dir, 'labelsTr_binary_all_classes'),
    binarize_segmentation
)

100%|██████████| 49/49 [03:24<00:00,  4.18s/it]


In [47]:
transform_label_vols_in_dir(
    file_paths_test, 
    os.path.join(data_dir, 'labelsTs_binary_all_classes'),
    binarize_segmentation
)

100%|██████████| 13/13 [00:37<00:00,  2.89s/it]


# Task 2: binary segmentation: only aneurysm segmentation vs background

In [49]:
transform_label_vols_in_dir(
    file_paths_train, 
    os.path.join(data_dir, 'labelsTr_binary_only_aneurysm'),
    lambda img: binarize_segmentation(img, only_specific_vessel_class=4)
)

100%|██████████| 49/49 [01:38<00:00,  2.00s/it]


In [50]:
transform_label_vols_in_dir(
    file_paths_test, 
    os.path.join(data_dir, 'labelsTs_binary_only_aneurysm'),
    lambda img: binarize_segmentation(img, only_specific_vessel_class=4)
)

100%|██████████| 13/13 [00:18<00:00,  1.39s/it]


# Task 3: 3 class segmentation, background, vessels, aneurysm

In [67]:
transform_label_vols_in_dir(
    file_paths_train, 
    os.path.join(data_dir, 'labelsTr_3_classes'),
    collapse_all_classes_except_aneurysm
)

100%|██████████| 49/49 [01:23<00:00,  1.69s/it]


In [68]:
transform_label_vols_in_dir(
    file_paths_test, 
    os.path.join(data_dir, 'labelsTs_3_classes'),
    collapse_all_classes_except_aneurysm
)

100%|██████████| 13/13 [00:23<00:00,  1.82s/it]
